In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

In [2]:
df = pd.read_csv('Sample-0004.csv')

In [3]:
df

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,1.433158,-0.135424,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
1,1.347939,-0.061015,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
2,1.441331,-0.080833,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
3,1.341744,-0.034491,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
4,1.441067,-0.074840,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34703,0.076511,-0.524380,-0.411514,-0.430135,0.457547,0.087674,0.005299,0.065032,1.169754,1.512796,...,1.369987,-0.120571,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7
34704,-2.564483,2.369977,-0.411514,-0.430135,0.294406,0.350849,1.676032,0.929084,0.992534,1.329547,...,-0.648481,3.051206,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7
34705,-2.564483,2.371282,-0.411514,-0.430135,1.316805,0.291929,0.108340,0.177398,0.992534,1.329547,...,-0.726665,0.336410,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7
34706,-2.564483,2.370748,-0.411514,-0.430135,0.110479,0.152485,0.436717,0.278139,0.992534,1.329547,...,-0.429432,0.336410,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7


In [4]:
X = df.drop(['Attack'],axis=1).values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)

In [6]:
features = df.dtypes[df.dtypes != 'object'].index

In [7]:
#Feature engineering
#Feature selection by information gain
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X_train, y_train)
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [8]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break        


In [9]:
X_fs = df[fs].values
X_fs.shape

(34708, 22)

In [10]:
#Re-split train & test sets after feature selection
X_train, X_test, y_train, y_test = train_test_split(X_fs,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)

In [11]:
X_train.shape

(27766, 22)

In [12]:
pd.Series(y_train).value_counts()

1    13447
5     6178
7     2740
0     1946
8     1210
9      784
2      651
6      366
3      238
4      206
dtype: int64

In [13]:
from collections import Counter
print("Before Sampling: ",Counter(y_train))

Before Sampling:  Counter({1: 13447, 5: 6178, 7: 2740, 0: 1946, 8: 1210, 9: 784, 2: 651, 6: 366, 3: 238, 4: 206})


In [14]:
from collections import Counter
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_trainf, y_trainf = smote_tomek.fit_resample(X_train, y_train)

In [15]:
print(sorted(Counter(y_trainf).items()))

[(0, 13398), (1, 13443), (2, 13400), (3, 12978), (4, 13276), (5, 12903), (6, 13431), (7, 13447), (8, 13408), (9, 13152)]


In [16]:
pd.Series(y_trainf).value_counts()

7    13447
1    13443
6    13431
8    13408
2    13400
0    13398
4    13276
9    13152
3    12978
5    12903
dtype: int64

In [17]:
%time
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
# define the Extra Trees model
et_model = ExtraTreesClassifier()

CPU times: total: 0 ns
Wall time: 0 ns


In [18]:
# define the parameter grid for hyperparameter tuning
%time
param_grid = {'n_estimators': [50, 100, 150, 200],
              'max_depth': [5, 10, 15, None],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              'bootstrap': [True, False]}

# define the stratified k-fold cross-validation object
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# perform hyperparameter tuning using grid search
et_grid = GridSearchCV(estimator=et_model, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
et_grid.fit(X_trainf, y_trainf)

# print the best hyperparameters
print("Best hyperparameters for Extra Trees: ", et_grid.best_params_)

CPU times: total: 0 ns
Wall time: 0 ns
Best hyperparameters for Extra Trees:  {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}


In [ ]:
# use the best hyperparameters to fit the Extra Trees model
import time

start_time = time.time()

# Your code block here
# %%time
et_model = ExtraTreesClassifier(**et_grid.best_params_)
et_model.fit(X_trainf, y_trainf)

end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time} seconds")

In [20]:
# evaluate the Extra Trees model on the test set
%time
et_accuracy = et_model.score(X_test, y_test)
print("Accuracy of Extra Trees model: {:.2f}%".format(et_accuracy*100))

CPU times: total: 0 ns
Wall time: 0 ns
Accuracy of Extra Trees model: 96.85%


In [21]:

%time
y_pred = et_model.predict(X_test)
print("Accuracy of Extra Trees: "+ str(accuracy_score(y_test, y_pred)))
print("Precision of Extra Trees: "+ str(precision_score(y_test, y_pred, average='weighted')))
print("Recall of Extra Trees: "+ str(recall_score(y_test, y_pred, average='weighted')))
print("Average F1 of Extra Trees: "+ str(f1_score(y_test, y_pred, average='weighted')))
print("F1 of CatBoost for each type of attack: "+ str(f1_score(y_test, y_pred, average=None)))
print(classification_report(y_test,y_pred))

CPU times: total: 0 ns
Wall time: 0 ns
Accuracy of Extra Trees: 0.9684528954191876
Precision of Extra Trees: 0.9722121093897398
Recall of Extra Trees: 0.9684528954191876
Average F1 of Extra Trees: 0.9698264915509724
F1 of CatBoost for each type of attack: [0.95238095 0.99910741 0.93939394 0.55405405 0.64347826 0.95655058
 0.92473118 0.99635834 0.9324547  0.83886256]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       487
           1       1.00      1.00      1.00      3362
           2       0.93      0.95      0.94       163
           3       0.47      0.68      0.55        60
           4       0.58      0.73      0.64        51
           5       0.98      0.93      0.96      1545
           6       0.91      0.95      0.92        91
           7       0.99      1.00      1.00       685
           8       0.93      0.94      0.93       302
           9       0.78      0.90      0.84       196

    accuracy                       

In [22]:
%time
y_pred_train =et_model.predict(X_trainf)
accuracy=accuracy_score(y_pred_train,y_trainf)
print(accuracy)

CPU times: total: 0 ns
Wall time: 0 ns
0.9999849438405252
